In [11]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [13]:
import getpass
import os
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = getpass.getpass()


model = ChatGroq(temperature=0, model_name="llama3-8b-8192")

 ········


In [15]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 15, 'total_tokens': 41, 'completion_time': 0.019886166, 'prompt_time': 0.00361199, 'queue_time': None, 'total_time': 0.023498156}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-c465b311-cc67-4457-b66f-75c5ed4b91f5-0')

In [17]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm just an AI, I don't have any information about your personal identity, including your name. I'm a new conversation each time you interact with me, so I don't retain any information about you or your previous conversations.", response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 15, 'total_tokens': 63, 'completion_time': 0.037324534, 'prompt_time': 0.004361164, 'queue_time': None, 'total_time': 0.041685698}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'stop', 'logprobs': None}, id='run-6470bbe9-7e76-4338-b9f7-d69c0283a280-0')

In [19]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='I think I can recall that your name is Bob!', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 40, 'total_tokens': 52, 'completion_time': 0.008908764, 'prompt_time': 0.016465447, 'queue_time': None, 'total_time': 0.025374211}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run-a0f078a1-a3ad-4561-962d-1d5720fb7317-0')

In [21]:
! pip3 install langchain-community

In [23]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [25]:
config = {"configurable": {"session_id": "abc2"}}

In [27]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

Parent run 2eea7a1f-1eb2-4d45-8361-6bdbe727fef1 not found for run 4f6ea867-5138-4ac9-afa7-02691ffa3a6e. Treating as a root run.


"Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [29]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 4674a9b7-7b70-4f8e-9850-57d229a7feb1 not found for run fc572117-184f-451a-b8a4-d1f9472d5195. Treating as a root run.


'I think I can recall... Your name is Bob, right?'

In [61]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [63]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm Bello Musa")]})

response.content

"Nice to meet you, Bello Musa! It's great to have you here. How can I assist you today? Do you have a specific question or topic you'd like to discuss? I'm all ears!"

In [65]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [67]:
config = {"configurable": {"session_id": "abc5"}}

In [69]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Abiola")],
    config=config,
)

response.content

Parent run fe09c1df-57a5-4523-bd58-a8b71b0f07e8 not found for run 0292b50b-f59e-4430-8252-4441cea7e209. Treating as a root run.


"Hello Abiola! It's nice to meet you! I'm here to help you with any questions or topics you'd like to discuss. What's on your mind today?"

In [71]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 7b6c0ddc-5d76-4d92-9a23-65a16acbd175 not found for run 6dc673ce-e8a8-409e-b45e-88b8587bdee4. Treating as a root run.


'Your name is Abiola!'

In [73]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [75]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡hola! ¡Hola Bob! ¿En qué puedo ayudarte hoy? (Hello! Hello Bob! How can I help you today?)'

In [77]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [79]:
config = {"configurable": {"session_id": "abc11"}}

In [81]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

Parent run 8d267e7b-cbd3-4978-a2c5-c4b0d6db1ceb not found for run 0be8a8a9-f1eb-4fa3-9667-25114e23fc83. Treating as a root run.


'¡hola! ¡Bienvenido, Todd! Soy tu asistente virtual. ¿En qué puedo ayudarte hoy? ¿Tienes alguna pregunta o necesitas ayuda con algo en particular?'

In [83]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)

response.content

Parent run 2626c4b7-0b72-4142-bcd8-acba90477ef3 not found for run 3fa04424-4bd0-423b-b07f-dc3b408f24d2. Treating as a root run.


'¡Ahah! ¡Claro que sí! Tu nombre es Todd, ¿recuerdas?'

In [85]:
from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=10):
    return messages[-k:]


chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [87]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [89]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I don't think you told me your name!"

In [91]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "language": "English",
    }
)
response.content

'you mentioned earlier that your favorite ice cream is vanilla!'

In [93]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [95]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run c1b5d361-80e6-43b3-a165-8e0bdbe776b9 not found for run 78d297f3-f59b-49df-81f4-127c35bf2bdb. Treating as a root run.


"I don't think you told me your name!"

In [97]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="whats my favorite ice cream?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run 7b984b92-c9ba-4965-bf33-35854af87155 not found for run 3db60b98-6ebb-479d-a283-ab930e394227. Treating as a root run.


"I don't know that either!"

In [99]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

Parent run 83b3ab40-c76d-4863-80f8-28c62d545069 not found for run c7d02775-c9a4-44fa-b63c-2edc6f7a48f3. Treating as a root run.


|Hi| Todd|!| Here|'s| one|:

|Why| couldn|'t| the| bicycle| stand| up| by| itself|?

|(W|ait| for| it|...)

|Because| it| was| two|-t|ired|!

|Hope| that| made| you| smile|!||